In [2]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# CREDENTIALS & CONFIG
API_URL = os.getenv("API_URL")
API_KEY = os.getenv("API_KEY")
DATA_DIR = "../data"

# Check if data directory exists
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
    print(f"[+] Created directory: {DATA_DIR}")

# LIST OF ALL KNOWN ACTION TYPES IN THE DATABASE
# Based on your Java AsyncDataWriter.java
ACTION_TYPES = [
    "RAID_INTERACTION",
    "BATTLE_END",
    "POKEMON_CAPTURED",
    "POKEMON_RELEASED",
    "POKEMON_HATCHED",
    "GTS_TRANSACTION",
    "SESSION_EVENT",  # Login / Logout / Join / Quit
    "SESSION_SNAPSHOT",  # Periodic player stats
    "COMMAND_USAGE",
    "PLAYER_DEATH"
]

print(f"[+] Configuration loaded. Target API: {API_URL}")

[+] Configuration loaded. Target API: http://node-marb.ponchisaohosting.xyz:25589/api/training-data


In [3]:
def download_and_save(action_type):
    print(f"[+] Requesting data for: {action_type}...")

    try:
        response = requests.get(
            API_URL,
            headers={"Authorization": f"Bearer {API_KEY}"},
            params={"action": action_type}
        )

        if response.status_code == 200:
            data = response.json()

            if not data:
                print(f"[-] No data found for {action_type}. Skipping.")
                return

            # Convert to DataFrame
            df = pd.DataFrame(data)

            # Construct filename: data/dataset_BATTLE_END_raw.csv
            filename = f"{DATA_DIR}/dataset_{action_type}_raw.csv"

            # Save Raw Data
            df.to_csv(filename, index=False)
            print(f"[+] Successfully saved {len(df)} rows to '{filename}'")

        else:
            print(f"[-] API Error {response.status_code}: {response.text}")

    except Exception as e:
        print(f"[-] Critical Error downloading {action_type}: {e}")

In [4]:
# ---------------------------------------------------------
# EXECUTE BATCH DOWNLOAD
# ---------------------------------------------------------
print("[INFO] Starting Batch Download...")

for action in ACTION_TYPES:
    download_and_save(action)

print("[INFO] Batch Download Complete.")

[INFO] Starting Batch Download...
[+] Requesting data for: RAID_INTERACTION...
[+] Successfully saved 54 rows to '../data/dataset_RAID_INTERACTION_raw.csv'
[+] Requesting data for: BATTLE_END...
[+] Successfully saved 457 rows to '../data/dataset_BATTLE_END_raw.csv'
[+] Requesting data for: POKEMON_CAPTURED...
[+] Successfully saved 115 rows to '../data/dataset_POKEMON_CAPTURED_raw.csv'
[+] Requesting data for: POKEMON_RELEASED...
[+] Successfully saved 45 rows to '../data/dataset_POKEMON_RELEASED_raw.csv'
[+] Requesting data for: POKEMON_HATCHED...
[-] No data found for POKEMON_HATCHED. Skipping.
[+] Requesting data for: GTS_TRANSACTION...
[+] Successfully saved 3 rows to '../data/dataset_GTS_TRANSACTION_raw.csv'
[+] Requesting data for: SESSION_EVENT...
[-] No data found for SESSION_EVENT. Skipping.
[+] Requesting data for: SESSION_SNAPSHOT...
[+] Successfully saved 1940 rows to '../data/dataset_SESSION_SNAPSHOT_raw.csv'
[+] Requesting data for: COMMAND_USAGE...
[+] Successfully save